In [171]:
import pandas as pd
from pandas import DataFrame, read_csv, Series
import numpy as np
import math
import sklearn.datasets
from sklearn.cluster import KMeans
import nltk
import re
from sklearn import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [172]:
# получение данных о студентах и их оценках по дисциплинам
import csv
def read_file(filename):
    f = open(filename)
    data = csv.reader(f)
    mentions = dict()
    for line in data:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [173]:
# косинусная мера сходства: насколько два пользователя похожи друг на друга. 
def dist_cosine(vecA, vecB):
    def dot_product(vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB: 
                d += vecA[dim]*vecB[dim]
        return d
    a = dot_product(vecA,vecB)
    b = math.sqrt(dot_product(vecA,vecA))
    c = math.sqrt(dot_product(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [174]:
def similar_students(studentID, studentsData, nBestStudents):
    matches = [(u, dist_cosine(studentsData[studentID], studentsData[u])) for u in studentsData.keys() if u != studentID]
    bestMatches = sorted(matches, key=lambda q:(q[1], q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    return [(x[0], x[1]) for x in bestMatches]

In [175]:
students_test = read_file("raw_data32.csv")
#df = DataFrame(students_test)
#df.columns.name = 'studentID'
#df.index.name = 'discipline'    
rec = similar_students('1', students_test, 10)


Most correlated with studentID: 1
StudentID: 9  Coeff: 0.7542152087381664
StudentID: 6  Coeff: 0.7248624347806895
StudentID: 7  Coeff: 0.7194365076928019
StudentID: 5  Coeff: 0.6837763094049188
StudentID: 4  Coeff: 0.5597650673256156
StudentID: 2  Coeff: 0.22271363971105235
StudentID: 3  Coeff: 0.06705017139158977
StudentID: 8  Coeff: 0


In [176]:
# Похожесть дисциплин
def read_subjects(filename):
    f = open(filename)
    data = csv.reader(f)
    subjects = dict()
    for line in data:
        ID = line[0]
        discipline = line[1]
        subjects[ID] = discipline
    f.close()
    return subjects

In [177]:
sd = read_subjects("subjects.csv")

# nltk.download() 

stopwords = set(stopwords.words("russian"))
#print(stopwords)


In [178]:
def review_to_words(review_text):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return(" ".join(meaningful_words))

In [179]:
d0 = pd.read_csv("subjects2.csv", encoding='utf-8')
d0.head(3)

,ID,Name,Abbr
0,1,Математическая логика и теория множеств,Мат. логика и ТМ
1,2,Математический анализ,Мат. Анализ
2,3,Алгебра и геометрия,Алг. и геом.


In [180]:
d1 = d0.copy()
d1['Name'] = d1['Name'].apply(lambda s: review_to_words(s))
d1.head(3)

,ID,Name,Abbr
0,1,математическая логика теория множеств,Мат. логика и ТМ
1,2,математический анализ,Мат. Анализ
2,3,алгебра геометрия,Алг. и геом.


In [181]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = False) 
vectorized = vectorizer.fit_transform(d1['Name'])
#print(vectorized)

num_samples, num_features = vectorized.shape
# print((num_samples, num_features)) # 918 дисциплин, 1123-мерный вектор признаков

vocabulary = vectorizer.get_feature_names()
#print(vocabulary[20])

num_clusters = 50
km = KMeans(n_clusters=num_clusters, init='random', n_init=10, verbose=1, random_state=3)
km.fit(vectorized)


Initialization complete
Iteration  0, inertia 2778.000
Iteration  1, inertia 2599.262
Iteration  2, inertia 2532.801
Iteration  3, inertia 2498.061
Converged at iteration 3: center shift 0.000000e+00 within tolerance 2.882799e-07
Initialization complete
Iteration  0, inertia 2729.000
Iteration  1, inertia 2524.008
Iteration  2, inertia 2480.079
Iteration  3, inertia 2448.783
Iteration  4, inertia 2440.390
Iteration  5, inertia 2440.121
Converged at iteration 5: center shift 0.000000e+00 within tolerance 2.882799e-07
Initialization complete
Iteration  0, inertia 2735.000
Iteration  1, inertia 2537.909
Iteration  2, inertia 2503.702
Iteration  3, inertia 2449.435
Iteration  4, inertia 2445.625
Iteration  5, inertia 2433.104
Iteration  6, inertia 2428.943
Iteration  7, inertia 2427.938
Iteration  8, inertia 2427.568
Converged at iteration 8: center shift 0.000000e+00 within tolerance 2.882799e-07
Initialization complete
Iteration  0, inertia 2771.000
Iteration  1, inertia 2520.597
Iterati

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300,
    n_clusters=50, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=3, tol=0.0001, verbose=1)

In [182]:
km.predict(vectorized)
km.cluster_centers_

array([[ 0.        ,  0.12244898,  0.        , ...,  0.        ,
         0.        ,  0.08163265],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [183]:
#print(km.cluster_centers_)
print(km.labels_)

[ 0  6  6 20 16  6  6  6  5  0  6  5  6  6  6  5  6  6  6  6  6  6  6  6 38
  6  6  6  6  6 25  0 10  8  6  6  6  6  6  6  6  6  6  6  6 25  6 20  6  6
  6  6  6  0 16  6  6 29  6 28  6  6  6 46  6  6  6 24  6 15 25  0  6 25  6
  6  6 16 28  6 16 25  6  6 16 15 21  6  6 25 24  6  6  6  6 21 25  6  0  6
 15  6 25  6  6  0  6  6 15  0 25 15  6  6 16  6  6  6  5 20  6  6  6  6  3
  4  4 16 16 19 19 22  6  0  6  6  6  6  6 31  6  4  6  6  6  6 15  6 38  6
  6  6  6 15  6  6  6  5  6  6  6  6  6  6  6  6  8  0  0  3  6  4  6 20 16
 16  0 17  6  5  6 17 17 17 17 17  6 17 17  6  6 17 17  6  6  6  6  6  6  6
  6 17  6 27  6 17  6  6 17 16  7  6  6  6  6  6  6 27  6 17  6 27  6  6  6
  0 10 10  6 25  6  6 34  6 24 27  6  6  6  4 16 25 19  0 25 19  6  6  6 25
 24 28 28 28  6  6  6 28 28  6 25 19  6  6 24  6  0  6  6  6 44 44 28 19 25
 25 28  6  6  6  6  6 31  3 19 24  6  6  6 46 18  6  6 19 46  6  6  4  6 25
  6 12  0  6  6 38 46  6 25 19  6  2 19 43  0 20  6 15 15 28  6  6  6  6  6
  6  6  6  6

In [187]:
d1['Name'][km.labels_ == 46]

63                          дополнительные главы алгебры
289           дополнительные главы исследования операций
294    научно исследовательский семинар дополнительны...
306         дополнительные главы функционального анализа
345                    дополнительные главы теории чисел
347      дополнительные главы дифференциальных уравнений
407           дополнительные главы математической физики
660      дополнительные главы теории экстремальных задач
700                         дополнительные главы анализа
707    дополнительные главы алгебры дискретной матема...
709                       дополнительные главы геометрии
864    дополнительные главы спектральной теории опера...
876    дополнительные главы спектральной теории опера...
Name: Name, dtype: object